## 제목을 입력합니다

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from pykrx import stock as st
import pickle

### csv 불러오기 실습

DataFrame : 2차원 데이터 -> table

In [2]:
stock_label = pickle.load(open('stock_label.sav', 'rb'))
stock_label.head(10)

,isin,num
0,095570,0
1,006840,1
2,027410,2
3,282330,3
4,138930,4
5,001460,5
6,001465,6
7,001040,7
8,079160,8
9,00104K,9


In [3]:
ohlcv = pd.read_csv('new_ohlcv최종.csv')
ohlcv.head(10)

,isin,s_date,open,high,low,close,volume,amount,updown
0,0,650,30900,36500,28000,35500,1994022,64841739950,14.89
1,0,651,33500,34700,31200,32400,392831,12926256800,-8.73
2,0,652,33050,33650,30950,31100,242730,7808534250,-4.01
3,0,653,31300,38800,31300,35450,703721,25144972800,13.99
4,0,654,36200,40300,34800,37850,499913,18764860300,6.77
5,0,655,38300,39200,35450,35700,191315,7088603600,-5.68
6,0,656,36300,36300,33050,33500,203023,6911408900,-6.16
7,0,657,34200,35250,33200,33250,149519,5137381200,-0.75
8,0,658,32600,36650,32000,36100,160592,5569432900,8.57
9,0,659,36150,38950,35300,37900,135862,5091580300,4.99


In [4]:
date_label = pickle.load(open('date_label.sav', 'rb'))
date_label.tail(10)

,s_date,num
2580,2023-06-27,2580
2581,2023-06-28,2581
2582,2023-06-29,2582
2583,2023-06-30,2583
2584,2023-07-03,2584
2585,2023-07-04,2585
2586,2023-07-05,2586
2587,2023-07-06,2587
2588,2023-07-07,2588
2589,2023-07-10,2589


In [5]:
grouped_df_list = []

for isin, group in ohlcv.groupby('isin'):
    group['y1'] = group['close'].shift(-1)
    group['y2'] = 0.5 * (group['updown'].shift(-1) / abs(group['updown'].shift(-1))) + 0.5
    group['y2'].fillna(0, inplace = True)
    group['y2'] = group['y2'].astype(int)
    group.dropna(axis=0, how='any', inplace=True)
    grouped_df_list.append(group)
    
train = pd.DataFrame([])
test = pd.DataFrame([])
for i, group in enumerate(grouped_df_list):
    if(i % 5 == 0):
        test = pd.concat([test, group])
    else:
        train = pd.concat([train, group])

y1_train = train.pop("y1")
y2_train = train.pop("y2")
x_train = train
y1_test = test.pop("y1")
y2_test = test.pop("y2")
x_test = test

In [6]:
model_close = LinearRegression()
model_close.fit(x_train, y1_train)
model_updown = LinearRegression()
model_updown.fit(x_train, y2_train)

LinearRegression()

In [7]:
pickle.dump(model_close, open('predict_close_model_new.sav', 'wb'))
pickle.dump(model_updown, open('predict_updown_model_new.sav', 'wb'))

In [8]:
today = "20230710" # sysdate로 변경
tmp_map = max(date_label['num'])
df = st.get_market_ohlcv(today, today, "005930").reset_index()
for s_date in df['날짜'].astype(str):
    if s_date not in date_label['s_date'].values:
        new_row = pd.DataFrame({'s_date': [s_date], 'num': [tmp_map + 1]})
        date_label = pd.concat([date_label, new_row], ignore_index=True)
        tmp_map += 1
pickle.dump(date_label, open('date_label.sav', 'wb'))

In [9]:
new_data = pd.DataFrame([])
date_set = date_label.tail(2)['s_date'].str.replace("-",'').to_list()
date_set_dash = date_label.tail(2)['s_date'].to_list()

for i, date in enumerate(date_set):
    df = st.get_market_ohlcv(date).reset_index()
    df["s_date"] = date_set_dash[i]
    df = df.dropna(axis=0, how='any')
    new_data = pd.concat([new_data,df])
new_data = new_data.rename({"티커":"isin", "시가" : "open", "고가" : "high", "저가" : "low", "종가" : "close", "거래량" : "volume", "거래대금" : "amount", "등락률" : "updown"}, axis=1)
new_data = new_data[['isin', 's_date', 'open', 'high', 'low', 'close', 'volume', 'amount', 'updown']].sort_values(['isin', 's_date'])
new_data['y1'] = new_data['close'].shift(-1)
new_data['y2'] = 0.5 * (new_data['updown'].shift(-1) / abs(new_data['updown'].shift(-1))) + 0.5
new_data.fillna(0, inplace = True)
new_data['y2'] = new_data['y2'].astype(int)
new_data = new_data[new_data["s_date"] != date_set_dash[-1]].reset_index()
new_data.drop(['index'], axis=1, inplace=True)
new_data['isin'] = new_data['isin'].replace(stock_label.set_index('isin')['num'])
new_data['s_date'] = new_data['s_date'].replace(date_label.set_index('s_date')['num'])
new_data

,isin,s_date,open,high,low,close,volume,amount,updown,y1,y2
0,340,2588,10300,10300,9990,10100,123131,1243830270,-1.94,9840.0,0
1,84,2588,574,574,550,554,244444,136866541,-3.48,558.0,1
2,188,2588,9610,9840,9510,9840,4498,43231440,1.44,9470.0,0
3,458,2588,72200,72500,71000,71000,4203,300015800,-1.66,70100.0,0
4,459,2588,54100,54100,52500,53100,1459,76921800,-1.85,53500.0,1
...,...,...,...,...,...,...,...,...,...,...,...
948,645,2588,310500,314000,304000,310000,142384,43876676000,-0.80,297000.0,0
949,309,2588,10100,10310,9950,10120,206195,2083608550,-1.08,9980.0,0
950,310,2588,9960,10300,9930,10100,466982,4713412080,1.51,9740.0,0
951,590,2588,2330,2450,2320,2420,1158393,2782227720,2.54,2490.0,1


In [10]:
y1_new_data = new_data.pop("y1")
y2_new_data = new_data.pop("y2")
today = new_data['close']
x_new_data = new_data

In [11]:
result = pd.concat([today, pd.DataFrame(model_close.predict(x_new_data), columns=['e_price']), y1_new_data , pd.DataFrame(model_updown.predict(x_new_data), columns=['e_updown']), y2_new_data], axis=1)
result['e_price'] = round(result['e_price'])
result['e_updown'] = round(result['e_updown'],1) + 0.5
result.rename({'y1':'r_price','y2':'r_updown'},axis=1, inplace=True)
result = result.astype(int)
result['e_rate'] = round((result['e_price'] - result['close']) / result['close'] * 100, 2)
result['r_rate'] = round((result['r_price'] - result['close']) / result['close'] * 100, 2)
result['등락 일치1'] = (result['e_updown']==result['r_updown']).astype(int)
result['등락 일치2'] = ((result['e_rate'] * result['r_rate']) / abs(result['e_rate'] * result['r_rate']) + 1) * 0.5
result = result.fillna(0)
result['등락 일치2'] = result['등락 일치2'].astype(int)
result['둘다 일치'] = ((result['등락 일치1'] == 1) & (result['등락 일치2'] == 1)).astype(int)
result['둘다 틀림'] = ((result['등락 일치1'] == 0) & (result['등락 일치2'] == 0)).astype(int)
result['1만 일치'] = ((result['등락 일치1'] == 1) & (result['등락 일치2'] == 0)).astype(int)
result['2만 일치'] = ((result['등락 일치1'] == 0) & (result['등락 일치2'] == 1)).astype(int)
result['error'] = round((abs((result['e_price'] - result['r_price']) / result['r_price']) * 100), 2)
result

,close,e_price,r_price,e_updown,r_updown,e_rate,r_rate,등락 일치1,등락 일치2,둘다 일치,둘다 틀림,1만 일치,2만 일치,error
0,10100,10098,9840,0,0,-0.02,-2.57,1,1,1,0,0,0,2.62
1,554,560,558,1,1,1.08,0.72,1,1,1,0,0,0,0.36
2,9840,9860,9470,0,0,0.20,-3.76,1,0,0,0,1,0,4.12
3,71000,70960,70100,0,0,-0.06,-1.27,1,1,1,0,0,0,1.23
4,53100,53049,53500,0,1,-0.10,0.75,0,0,0,1,0,0,0.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,310000,309731,297000,0,0,-0.09,-4.19,1,1,1,0,0,0,4.29
949,10120,10126,9980,0,0,0.06,-1.38,1,0,0,0,1,0,1.46
950,10100,10121,9740,0,0,0.21,-3.56,1,0,0,0,1,0,3.91
951,2420,2441,2490,0,1,0.87,2.89,0,1,0,0,0,1,1.97


In [12]:
# pd.set_option('display.max_rows', None)
result

,close,e_price,r_price,e_updown,r_updown,e_rate,r_rate,등락 일치1,등락 일치2,둘다 일치,둘다 틀림,1만 일치,2만 일치,error
0,10100,10098,9840,0,0,-0.02,-2.57,1,1,1,0,0,0,2.62
1,554,560,558,1,1,1.08,0.72,1,1,1,0,0,0,0.36
2,9840,9860,9470,0,0,0.20,-3.76,1,0,0,0,1,0,4.12
3,71000,70960,70100,0,0,-0.06,-1.27,1,1,1,0,0,0,1.23
4,53100,53049,53500,0,1,-0.10,0.75,0,0,0,1,0,0,0.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,310000,309731,297000,0,0,-0.09,-4.19,1,1,1,0,0,0,4.29
949,10120,10126,9980,0,0,0.06,-1.38,1,0,0,0,1,0,1.46
950,10100,10121,9740,0,0,0.21,-3.56,1,0,0,0,1,0,3.91
951,2420,2441,2490,0,1,0.87,2.89,0,1,0,0,0,1,1.97


In [28]:
model_close.score(x_new_data, y1_new_data)

0.9993189060906527

In [30]:
model_updown.score(x_new_data, round(y2_new_data,1) + 0.5)

-1.0315786980504047

In [13]:
result['error'].max()

83.96

In [14]:
result['error'].min()

0.0

In [15]:
available = result[(result['error'] <= 5) & (result['둘다 일치'] == 1)]

In [16]:
str(round(len(result[result['error'] <= 5])/len(result) * 100,2)) + "%"

'95.7%'

In [17]:
str(round(len(result[result['등락 일치1'] == 1])/len(result) * 100,2)) + "%"

'56.56%'

In [18]:
str(round(len(result[result['등락 일치2'] == 1])/len(result) * 100,2)) + "%"

'42.39%'

In [19]:
str(round(len(result[result['둘다 일치'] == 1])/len(result) * 100,2)) + "%"

'18.99%'

In [20]:
str(round(len(result[result['둘다 틀림'] == 1])/len(result) * 100,2)) + "%"

'20.04%'

In [21]:
str(round(len(result[result['1만 일치'] == 1])/len(result) * 100,2)) + "%"

'37.57%'

In [22]:
str(round(len(result[result['2만 일치'] == 1])/len(result) * 100,2)) + "%"

'23.4%'

In [23]:
str(round(len(result[result['둘다 일치'] == 1])/len(result[result['등락 일치1'] == 1]) * 100,2)) + "%"

'33.58%'

In [24]:
str(round(len(result[result['1만 일치'] == 1])/len(result[result['등락 일치1'] == 1]) * 100,2)) + "%"

'66.42%'

In [25]:
str(round(len(result[result['둘다 일치'] == 1])/len(result[result['등락 일치2'] == 1]) * 100,2)) + "%"

'44.8%'

In [26]:
str(round(len(result[result['2만 일치'] == 1])/len(result[result['등락 일치2'] == 1]) * 100,2)) + "%"

'55.2%'

In [27]:
str(round(len(available)/len(result) * 100,2)) + "%"

'18.47%'